# Étape 3 : Entraînement et Test de la Cascade de Traduction

Ce notebook permet de lancer l'entraînement ASR (Speech-to-Text) sur CPU et de tester la chaîne de traduction finale (Mina ➔ Éwé ➔ Français).

In [1]:
import sys
import importlib
from pathlib import Path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))

import pandas as pd
import torch
import torchaudio
from transformers import WhisperForConditionalGeneration, WhisperProcessor, AutoTokenizer, AutoModelForSeq2SeqLM, MarianTokenizer, MarianMTModel
from src.config import settings
import src.models.train_whisper_cpu

# Reload de nos modules persos
import src.models.translation_mina_ewe
import src.models.translation_ewe_fr
import src.pipeline.translate_cascade
importlib.reload(src.models.train_whisper_cpu)
importlib.reload(src.models.translation_mina_ewe)
importlib.reload(src.models.translation_ewe_fr)
importlib.reload(src.pipeline.translate_cascade)

from src.models.train_whisper_cpu import train_whisper_on_cpu
from src.pipeline.translate_cascade import TranslationCascade

c:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Entraînement ASR (Whisper sur CPU)

 (> **Note :** L'entraînement sur CPU est lent. Nous utilisons un gel des couches (freeze) à 90% pour accélérer le processus. Les hyperparamètres sont configurés dans `src/config/settings.py`.

In [2]:
# Chargement du dataset préparé à l'étape 2
dataset_path = settings.PROCESSED_DIR / "bible_asr_dataset.csv"

if dataset_path.exists():
    df = pd.read_csv(dataset_path)
    print(f"Dataset chargé : {len(df)} exemples.")
    
    # Lancement de l'initialisation de l'entraînement
    # Note: assurez-vous d'avoir assez de RAM
    model, processor = train_whisper_on_cpu(None) 
    
    # --- SAUVEGARDE DU MODÈLE ENTRAÎNÉ (EXPORT) ---
    export_dir = settings.PROJECT_ROOT / "models" / "whisper-ewe-mina-final"
    print(f"Exportation du modèle ASR vers {export_dir}...")
    model.save_pretrained(export_dir)
    processor.save_pretrained(export_dir)
    print("Modèle ASR exporté avec succès !")
else:
    print("Erreur : Le dataset ASR n'a pas encore été généré. Veuillez exécuter le notebook 02 d'abord.")

Dataset chargé : 23559 exemples.
--- Initialisation Fine-tuning Whisper (openai/whisper-base) sur CPU (12 coeurs) ---
Chargement depuis C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\data\processed\bible_asr_dataset.csv...


Generating train split: 23559 examples [00:00, 174983.94 examples/s]


Pré-traitement du dataset (Feature Extraction)...


Map (num_proc=10):   0%|          | 0/23559 [00:29<?, ? examples/s]


RuntimeError: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, 7, and 8. On Windows, ensure you've installed
             the "full-shared" version which ships DLLs.
          2. The PyTorch version (2.10.0+cpu) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\venv\Lib\site-packages\torchcodec\libtorchcodec_core8.dll
FFmpeg version 7: Could not load this library: C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\ve

## 2. Exportation des autres modèles (NLLB et Opus-MT)

Pour avoir une solution 100% autonome, nous téléchargeons et sauvegardons localement les modèles de traduction.

In [ ]:
print("Téléchargement et export des modèles de traduction...")

# 1. Export NLLB (Mina -> Ewe)
nllb_model_name = "facebook/nllb-200-distilled-600M"
nllb_path = settings.PROJECT_ROOT / "models" / "nllb-mina-ewe-local"

if not nllb_path.exists():
    print(f"Téléchargement de {nllb_model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name)
    
    print(f"Sauvegarde dans {nllb_path}...")
    model.save_pretrained(nllb_path)
    tokenizer.save_pretrained(nllb_path)
else:
    print(f"Modèle NLLB déjà présent dans {nllb_path}")

# 2. Export Opus-MT (Ewe -> Français)
opus_model_name = "Helsinki-NLP/opus-mt-ee-fr"
opus_path = settings.PROJECT_ROOT / "models" / "opus-ewe-fr-local"

if not opus_path.exists():
    print(f"Téléchargement de {opus_model_name}...")
    tokenizer = MarianTokenizer.from_pretrained(opus_model_name)
    model = MarianMTModel.from_pretrained(opus_model_name)
    
    print(f"Sauvegarde dans {opus_path}...")
    model.save_pretrained(opus_path)
    tokenizer.save_pretrained(opus_path)
else:
    print(f"Modèle Opus déjà présent dans {opus_path}")

print("Tous les modèles sont prêts localement !")

Téléchargement et export des modèles de traduction...
Modèle NLLB déjà présent dans C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\nllb-mina-ewe-local
Modèle Opus déjà présent dans C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\opus-ewe-fr-local
Tous les modèles sont prêts localement !


## 3. Test Audio Complet avec les 3 Modèles Exportés (Local)

Nous chargeons maintenant **les 3 modèles** uniquement depuis le dossier `models/`.

In [ ]:
# Chemins locaux
asr_path = settings.PROJECT_ROOT / "models" / "whisper-ewe-mina-final"
nllb_path = settings.PROJECT_ROOT / "models" / "nllb-mina-ewe-local"
opus_path = settings.PROJECT_ROOT / "models" / "opus-ewe-fr-local"

print("1. Chargement du modèle ASR local...")
try:
    loaded_processor = WhisperProcessor.from_pretrained(asr_path)
    loaded_model = WhisperForConditionalGeneration.from_pretrained(asr_path)
    print("   OK.")
except Exception as e:
    print(f"   Erreur ASR (il faut lancer l'entraînement avant) : {e}")
    # Fallback pour ne pas bloquer si pas entraîné dans cette session
    loaded_processor = processor 
    loaded_model = model

print("2. Initialisation de la Cascade avec les modèles locaux...")
cascade = TranslationCascade(nllb_path=str(nllb_path), opus_path=str(opus_path))
print("   OK.")


1. Chargement du modèle ASR local...


INFO:src.pipeline.translate_cascade:Initialisation de la cascade de traduction...
INFO:src.models.translation_mina_ewe:Chargement du modèle Mina-Ewe : C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\nllb-mina-ewe-local


   OK.
2. Initialisation de la Cascade avec les modèles locaux...


The tokenizer you are loading from 'C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\nllb-mina-ewe-local' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
INFO:src.models.translation_ewe_fr:Chargement du modèle Transformers C:\EPL\MTH2321\MTH_2321_APEKE\Projet_traduction_ewe_francais\models\opus-ewe-fr-local


   OK.


In [ ]:
# --- INFERENCE ---
import soundfile as sf
import scipy.signal
import numpy as np
import torch

sample_audio = settings.PROCESSED_DIR / "audio_16k" / "gegbe_gen_01.wav"

if sample_audio.exists():
    print(f"\nTraitement audio : {sample_audio.name}")
    
    waveform, sr = sf.read(sample_audio)
    
    print(f"Durée originale : {len(waveform) / sr:.2f} secondes")
    print(f"Échantillons    : {len(waveform):,}")
    print(f"Sample rate lu  : {sr} Hz")
    
    # Mono si stéréo
    if len(waveform.shape) > 1:
        waveform = waveform.mean(axis=1)
        
    # Resample si besoin
    if sr != 16000:
        num_samples = int(round(len(waveform) * 16000 / sr))
        waveform = scipy.signal.resample(waveform, num_samples)
    
    # Préparation Whisper
    input_features = loaded_processor(
        waveform,
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features
    
    # Très important : déplacer sur le bon device (GPU si disponible)
    device = next(loaded_model.parameters()).device
    input_features = input_features.to(device)
    
    with torch.no_grad():
        predicted_ids = loaded_model.generate(
            input_features,
            language=None,                     # ou "mina" / None si détection auto
            task="transcribe",
            return_timestamps=True,             # active le découpage automatique 30s + stitching
            condition_on_prev_tokens=True,
            temperature=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],  # fallback si répétitions/hallucinations
            compression_ratio_threshold=1.35,
            logprob_threshold=-1.0,
            no_speech_threshold=0.6,
        )
    
    transcription = loaded_processor.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0]
    
    print(f"\nTranscription (ASR) :\n{transcription}\n")
    
    final_result = cascade.translate_mina_to_french(transcription)
    
    print("=== RÉSULTAT 100% LOCAL ===")
    print(f"TEXTE (ASR) : {final_result['mina']}")
    print(f"PIVOT (EWE) : {final_result['ewe']}")
    print(f"TRAD (FR)   : {final_result['french']}")

else:
    print("Fichier audio non trouvé.")


Traitement audio : gegbe_gen_01.wav
Durée originale : 368.40 secondes
Échantillons    : 5,894,400
Sample rate lu  : 16000 Hz


INFO:src.pipeline.translate_cascade:Source (Mina): Tɔhonɔ gblɔ na Mosè ku Arɔn be Izràɛlviwo



Transcription (ASR) :
 Gomejdejbehoma Eta tutu guan Hihiabe dodo So gomejdejbehma Mauro d'yunku siku aniban Aniwa leñamaa Eibeleqbalo Viviti tri tri porla gomejdejbea pubajah Va maubhe bongboa di nasa leciojis Maughe bongbi



INFO:src.pipeline.translate_cascade:Pivot (Ewe): Tɔhonɔ gblɔ na Mose kple Aron be woanye Israelviwo
INFO:src.pipeline.translate_cascade:Cible (Français): Moïse et Aaron ont reçu l'ordre d'être des Israélites.


=== RÉSULTAT 100% LOCAL ===
TEXTE (ASR) : Tɔhonɔ gblɔ na Mosè ku Arɔn be Izràɛlviwo
PIVOT (EWE) : Tɔhonɔ gblɔ na Mose kple Aron be woanye Israelviwo
TRAD (FR)   : Moïse et Aaron ont reçu l'ordre d'être des Israélites.


In [ ]:
import sounddevice as sd
import numpy as np

# --- CONFIGURATION MICRO ---
fs = 16000  # Fréquence d'échantillonnage requise par Whisper
duration = 5  # Durée de l'enregistrement en secondes (à ajuster)

print(f"\n--- ATTENTION : Enregistrement de {duration} secondes ---")
print("Parlez maintenant...")

# Capture de l'audio
# sd.rec enregistre dans un array NumPy
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
sd.wait()  # Attend que l'enregistrement soit terminé

print("Enregistrement terminé. Traitement en cours...\n")

# Conversion en mono (déjà fait par channels=1, mais on aplatit l'array)
waveform = recording.flatten()

# --- REPRISE DE TON PROCESSUS INFERENCE ---

# Préparation Whisper (plus besoin de resample si fs=16000)
input_features = loaded_processor(
    waveform,
    sampling_rate=fs,
    return_tensors="pt"
).input_features

# Déplacement sur le bon device
device = next(loaded_model.parameters()).device
input_features = input_features.to(device)

with torch.no_grad():
    predicted_ids = loaded_model.generate(
        input_features,
        language=None, # Auto-détection
        task="transcribe",
        return_timestamps=True,
        condition_on_prev_tokens=True,
        temperature=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
        compression_ratio_threshold=1.35,
        logprob_threshold=-1.0,
        no_speech_threshold=0.6,
    )

transcription = loaded_processor.batch_decode(
    predicted_ids,
    skip_special_tokens=True
)[0]

# --- TRADUCTION ---
if transcription.strip():
    final_result = cascade.translate_mina_to_french(transcription)
    
    print("=== RÉSULTAT 100% LOCAL (MICRO) ===")
    print(f"TEXTE (ASR) : {final_result['mina']}")
    print(f"PIVOT (EWE) : {final_result['ewe']}")
    print(f"TRAD (FR)   : {final_result['french']}")
else:
    print("Aucun son détecté ou transcription vide.")


--- ATTENTION : Enregistrement de 5 secondes ---
Parlez maintenant...
Enregistrement terminé. Traitement en cours...



INFO:src.pipeline.translate_cascade:Source (Mina):  Apitonila
INFO:src.pipeline.translate_cascade:Pivot (Ewe): Apitonila
INFO:src.pipeline.translate_cascade:Cible (Français): Aptonaila


=== RÉSULTAT 100% LOCAL (MICRO) ===
TEXTE (ASR) :  Apitonila
PIVOT (EWE) : Apitonila
TRAD (FR)   : Aptonaila
